## **Trabajo Práctico Final Grupal: Bank Subscription**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre: Candela
- Apellido: Abatedaga
- Legajo: 1673117



- Nombre: Martín
- Apellido: Aiscar
- Legajo: 1635487

**Entrega del trabajo: Viernes 25 de noviembre a las 23h59.**

##**Pipeline de Machine Learning**

In [ ]:
## Dividimos el dataset en la variable dependiente "y", en este caso "Subscription", y las independientes "x".
y = np.array(bank_ds[["Subscription"]])
x = bank_ds.drop(['Subscription'], axis=1)

In [ ]:
## Separamos el dataset en train y test. Corresponderán a test el 30% de las muestras.
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=4)

In [ ]:
## Imprimimos las dimensiones del set de train y del de test.
print(f'Dimensiones del set de train: {xtrain.shape}')
print(f'Dimensiones del set de test: {xtest.shape}')

In [ ]:
## Diferenciamos las variables numéricas de las categóricas.
numeric_features = ['Age','Balance (euros)','Last Contact Day','Last Contact Month','Last Contact Duration','Campaign','Pdays','Previous']

variables_cat= ['Job','Marital Status','Education','Credit','Housing Loan','Personal Loan']

In [ ]:
## Definimos la transformación a realizar para las variables numéricas. 
## Las estandarizaremos utilizando StandardScaler.
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

## Definimos la transformación a realizar para las variables categóricas.
## Generaremos dummies utilizado OneHotEncoder.
transformacion_cat = OneHotEncoder(handle_unknown="error", sparse=False)

In [ ]:
## Juntamos las transformaciones definidas previamente.
## Definimos el ColumnTransformer que será ejecutado al momento del fitting.
preprocesamiento = ColumnTransformer(
    transformers=[ 
        ("num", numeric_transformer, numeric_features),
        ("cat", transformacion_cat, variables_cat),
    ]
)

In [ ]:
## Creamos un Pipeline para aplicar secuencialmente la lista de transformaciones definida previamente y un estimador final.
pipeline1 = Pipeline(
    steps=[("preprocesamiento",preprocesamiento),("estimador",SVC())]
)

In [ ]:
## Definimos los hiperparámetros de los modelos a comparar.
parametros = [
    {
        "estimador": (LogisticRegression(),),
          "estimador__C": (0.001,1,10)
    },
    {
        "estimador": (SVC(),),
          "estimador__kernel":('linear', 'rbf'), 
          "estimador__C":(0.1,1, 10), 
          "estimador__gamma":(0.001, 0.01, 0.1,1, 10)
          "estimador__probability":("True")          
    }
]

In [ ]:
## Creamos el Grid Search + Cross Validation.
grid_search1 = GridSearchCV(pipeline1, parametros,
                  refit = True,
                   cv = 5,
                   verbose=40)
grid_search1.fit(xtrain,ytrain.ravel())

In [ ]:
## Imprimimos cuál fue el mejor modelo junto con los mejores hiperparámetros.
print("Los mejores parámetros fueron %s con un score de %0.4f" % (grid_search1.best_params_, grid_search1.best_score_))

In [ ]:
## Realizamos las predicciones sobre las muestras de test.
ypred1 = grid_search1.predict(xtest)
print(ypred1)

In [ ]:
## Computamos el accuracy.
test_acc = accuracy_score(ytest, ypred1)
print("El accuracy es " + str(test_acc))

In [ ]:
## Computamos el área debajo de la curva ROC (AUC).
yproba1 = grid_search1.predict_proba(xtest)
fpr1, tpr1, thresholds = roc_curve(ytest.astype(int), yproba1[:,1], drop_intermediate = False, pos_label=2)
auc_value = auc(fpr1, tpr1)
print("El AUC es = " + str(auc_value))

In [ ]:
## Calculamos e imprimimos la matriz de confusión.
cm = confusion_matrix(ytest, ypred1)
df_cm = pd.DataFrame(cm, index = ['No accede', 'Accede'], columns = ['No accede', "Accede"])
plt.figure(figsize = (6,4))
sns.heatmap(df_cm, annot=True,fmt='g')
plt.title('Matriz de Confusión')
plt.show()

##**Analisis de Componentes Pricipales**

In [ ]:
## Definimos la proporción de variación que queremos explicar con los componentes a extraer.
n_comps = 0.8

## Definimos el PCA.
pca = PCA(n_components = n_comps)

## Definimos las transformaciones a realizar para las variables numéricas. 
## Las estandarizaremos utilizando StandardScaler y luego aplicamos el método PCA.
numeric_transformer1 = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler()), ("pca", pca)]
)

## Juntamos las transformaciones definidas previamente.
## Definimos el ColumnTransformer que será ejecutado al momento del fitting.
preprocesamiento1 = ColumnTransformer(
    transformers=[ 
        ("num", numeric_transformer1, numeric_features),
        ("cat", transformacion_cat, variables_cat),
    ]
)

## Definimos un nuevo pipeline que incluye el método PCA en el preprosemiento.
pipeline2=Pipeline(
    steps=[('preprocesamiento',preprocesamiento1),('estimador',SVC())]
)

In [ ]:
## Creamos el Grid Search + Cross Validation. 
grid_search2 = GridSearchCV(pipeline2, parametros,
                  refit = True, ## Refit nos devuelve el modelo con los mejores parametros encontrados.
                   cv = 5, ## Indica la cantidad de folds.
                   verbose=40)
## Entrenamos.
grid_search2.fit(xtrain,ytrain.ravel())

In [ ]:
## Imprimimos cuál fue el mejor modelo y la combinación de hiper parámetros.
print("Los mejores parametros fueron %s con un score de %0.2f" % (grid_search2.best_params_, grid_search2.best_score_))

In [ ]:
## Realizamos las predicciones sobre las muestras de test.
ypred2 = grid_search2.predict(xtest)
print(ypred2)

In [ ]:
## Computamos el accuracy.
test_acc = accuracy_score(ytest, ypred2)
print("El accuracy es " + str(test_acc))

In [ ]:
## Computamos el AUC ROC.
yproba2 = grid_search2.predict_proba(xtest)
fpr1, tpr1, thresholds = roc_curve(ytest.astype(int), yproba2[:,1], drop_intermediate = False, pos_label=2)
auc_value = auc(fpr1, tpr1)
print("El AUC es = " + str(auc_value))

In [ ]:
## Calculamos e imprimimos la matriz de confusión.
cm = confusion_matrix(ytest, ypred2)
df_cm = pd.DataFrame(cm, index = ['No accede', 'Accede'], columns = ['No accede', "Accede"])
plt.figure(figsize = (6,4))
sns.heatmap(df_cm, annot=True,fmt='g')
plt.title('Matriz de Confusión con PCA')
plt.show()